In [2]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow import keras

In [3]:
train_data = pd.read_csv("Data/train.csv")

target = train_data['label']
train_vars = train_data.drop(['label'],axis=1)

X_train = train_vars/255
y = target

In [4]:
X_train = X_train.values.reshape(X_train.shape[0],28,28,1)

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(64, (4,4),activation='relu',input_shape=(28,28,1)))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(32, (7,7),activation='relu'))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(32, (2,2),activation='relu'))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))

model.add(layers.Flatten())
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 25, 25, 64)        1088      
_________________________________________________________________
dropout (Dropout)            (None, 25, 25, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 32)          100384    
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 32)          0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 32)          4

In [6]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=8)
lr_reduction = ReduceLROnPlateau(monitor='val_loss',patience=5,verbose=1,factor=0.2)

In [7]:
model.fit(X_train,np.array(y),
          epochs=50,validation_split=0.2,
         batch_size=128, shuffle=True,callbacks =[lr_reduction,es])

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 33s 689us/sample - loss: 0.3083 - accuracy: 0.9044 - val_loss: 0.0760 - val_accuracy: 0.9816
Epoch 2/50
48000/48000 [==============================] - 33s 679us/sample - loss: 0.0813 - accuracy: 0.9756 - val_loss: 0.0501 - val_accuracy: 0.9843
Epoch 3/50
48000/48000 [==============================] - 33s 691us/sample - loss: 0.0561 - accuracy: 0.9829 - val_loss: 0.0413 - val_accuracy: 0.9883
Epoch 4/50
48000/48000 [==============================] - 34s 701us/sample - loss: 0.0436 - accuracy: 0.9870 - val_loss: 0.0318 - val_accuracy: 0.9896
Epoch 5/50
48000/48000 [==============================] - 32s 660us/sample - loss: 0.0363 - accuracy: 0.9884 - val_loss: 0.0493 - val_accuracy: 0.9860
Epoch 6/50
48000/48000 [==============================] - 32s 674us/sample - loss: 0.0310 - accuracy: 0.9900 - val_loss: 0.0258 - val_accuracy: 0.9915
Epoch 7/50
48000/48000 [====================

In [8]:
test_data = pd.read_csv('Data/test.csv')
ImageId = test_data['id']
NN_test = test_data.drop(['id'],axis=1)
NN_test = NN_test/255
NN_test = NN_test.values.reshape(NN_test.shape[0],28,28,1)

predictions = model.predict_classes(NN_test)

In [ ]:
sub = pd.DataFrame({'id':ImageId, 'label':predictions})
sub.to_csv("submission.csv",index=False)